# Capstone project: Final assignment week 4 

## Introduction/Business problem

An US-based company has regular work travels in Rome, Naples, and Florence (Italy).

The company has already found the accomodations for these recurrent stays and would like to know which are the best-rated pizza places where their employers cold have dinner

**The objective of the study is to find a good trade-off between pizzeria-rating and closeness to the already chosen accomodations**

The maximum distance the employers should walk from their pre-selected accomodation is 500m

## Data Section

Data given by the companies are the three addresses of the three accomodations. 

Rome: 33 Via di Sant'Agata de Goti, Roma, RM
Nample: 133, Via Stella, Napoli, NA
Florence: 3, Via Del Leone, Firenze, FI

I leverage Foursquare data to get the locations and proceed with the analysis

### Visualization of the locations with Folium

In [1]:
import pandas as pd
import numpy as np
import requests # library to handle requests
from bs4 import BeautifulSoup
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = 'O4YSARF1VKMD3JXMQCZYNAHTJCV3LHDB3DYMETHZXAAJMC0I' # your Foursquare ID
CLIENT_SECRET = '0POWTMO1RCRDOJ4VGTKE4QJ5PCPNIRIBH5YCG4NX5LKW2V1W' # your Foursquare Secret
ACCESS_TOKEN = 'C1TYPH4VKGU4T5LFZSTF3CC34N3R2OSQZHGYOEF5AFTDTDOY' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O4YSARF1VKMD3JXMQCZYNAHTJCV3LHDB3DYMETHZXAAJMC0I
CLIENT_SECRET:0POWTMO1RCRDOJ4VGTKE4QJ5PCPNIRIBH5YCG4NX5LKW2V1W


### Some useful functions

In [3]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [4]:
def get_congitute_latitude_accomodation(address):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude

In [5]:
def filter_data(dataframe):
    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

    return dataframe_filtered

In [6]:
def query_into_dataframe(CLIENT_ID, CLIENT_SECRET, address,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT):
    latitude, longitude = get_congitute_latitude_accomodation(address)
    
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    
    # assign relevant part of JSON to venues
    venues = results['response']['venues']

    # tranform venues into a dataframe
    dataframe = pd.json_normalize(venues)   
    return dataframe, latitude, longitude

In [11]:
def plot_map(latitude, longitude, dataframe_filtered):
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the Conrad Hotel

    # add a red circle marker to represent the place rented
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        color='red',
        popup='Rented place',
        fill = True,
        fill_color = 'red',
        fill_opacity = 0.6
    ).add_to(venues_map)

    # add the Italian restaurants as blue circle markers
    for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='blue',
            popup=label,
            fill = True,
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

    # display map
    return venues_map

#### Visualise Rome address and closeby pizza locations

In [12]:
address = "33 Via di Sant'Agata de Goti, Roma, RM"
search_query = 'Pizza'
radius = 500 #500 meters
df, latitude, longitude = query_into_dataframe(CLIENT_ID, CLIENT_SECRET, address,ACCESS_TOKEN, VERSION, search_query,radius,LIMIT)
df = filter_data(df)
plot_map(latitude, longitude, df)

#### Visualise Naples address and closeby pizza locations

In [13]:
address = '133, Via Stella, Napoli, NA'
search_query = 'Pizza'
radius = 500 #500 meters
df, latitude, longitude = query_into_dataframe(CLIENT_ID, CLIENT_SECRET, address,ACCESS_TOKEN, VERSION, search_query,radius,LIMIT)
df = filter_data(df)
plot_map(latitude, longitude, df)

KeyError: "None of [Index(['name', 'categories', 'id'], dtype='object')] are in the [columns]"

**There are no pizza places in 500 meters, we redo the analysis in 1000 meters**

In [16]:
address = '133, Via Stella, Napoli, NA'
search_query = 'Pizza'
radius = 1000 #500 meters
df, latitude, longitude = query_into_dataframe(CLIENT_ID, CLIENT_SECRET, address,ACCESS_TOKEN, VERSION, search_query,radius,LIMIT)
df = filter_data(df)
plot_map(latitude, longitude, df)

#### Visualise Florence address and closeby pizza locations

In [15]:
address = '3, Via Del Leone, Firenze, FI'
search_query = 'Pizza'
radius = 500 #500 meters
df, latitude, longitude = query_into_dataframe(CLIENT_ID, CLIENT_SECRET, address,ACCESS_TOKEN, VERSION, search_query,radius,LIMIT)
df = filter_data(df)
plot_map(latitude, longitude, df)

#### Let's again assume that you are staying at the Conrad hotel. So let's start by converting the Contrad Hotel's address to its latitude and longitude coordinates.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.


In [37]:
address = "33 Via di Sant'Agata de Goti, Roma, RM"
address = '133, Via Stella, Napoli, NA'
address = '3, Via Del Leone, Firenze, FI'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.7694639 11.2430554


In [38]:
search_query = 'Pizza'
radius = 1000
print(search_query + ' .... OK!')


Pizza .... OK!


In [39]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=O4YSARF1VKMD3JXMQCZYNAHTJCV3LHDB3DYMETHZXAAJMC0I&client_secret=0POWTMO1RCRDOJ4VGTKE4QJ5PCPNIRIBH5YCG4NX5LKW2V1W&ll=43.7694639,11.2430554&oauth_token=C1TYPH4VKGU4T5LFZSTF3CC34N3R2OSQZHGYOEF5AFTDTDOY&v=20180604&query=Pizza&radius=1000&limit=30'

In [40]:
results = requests.get(url).json()
#results

In [41]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4b9d3668f964a520f99936e3,Gusta Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1613129402,False,Via Maggio 46r,Sdrucciolo dei Pitti,43.766584,11.248758,"[{'label': 'display', 'lat': 43.76658430724039...",559,50125,IT,Oltrarno,Firenze,Toscana,Italia,"[Via Maggio 46r (Sdrucciolo dei Pitti), 50125 ...",NaN
1,59e9b5d749281462d77af718,Al 58 Self-Service Risto Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1613129402,False,"Via Il Prato, 58",NaN,43.776610,11.241200,"[{'label': 'display', 'lat': 43.77661, 'lng': ...",809,50123,IT,NaN,Firenze,Toscana,Italia,"[Via Il Prato, 58, 50123 Firenze Toscana, Italia]",NaN
2,536143f2498eca9473812969,Mangia Pizza Firenze,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1613129402,False,"Via Lambertesca, 24",Via Por Santa Maria,43.768940,11.253940,"[{'label': 'display', 'lat': 43.76894, 'lng': ...",876,50122,IT,NaN,Firenze,Toscana,Italia,"[Via Lambertesca, 24 (Via Por Santa Maria), 50...",96100203
3,4db302806e8179a9135f3687,Il Borgo Della Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1613129402,False,Borgo San Frediano,NaN,43.770023,11.242569,"[{'label': 'display', 'lat': 43.77002309454146...",73,NaN,IT,NaN,Firenze,Toscana,Italia,"[Borgo San Frediano, Firenze Toscana, Italia]",NaN
4,5054c601e4b0cbb61a802454,Pizza Bike,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1613129402,False,NaN,NaN,43.774117,11.242652,"[{'label': 'display', 'lat': 43.77411724254026...",519,NaN,IT,NaN,NaN,NaN,Italia,[Italia],NaN


In [42]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#dataframe_filtered

In [43]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the place rented
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Rented place',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<a id="item2"></a>


## 2. Explore a Given Venue

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**


### A. Let's explore the closest Italian restaurant -- _Harry's Italian Pizza Bar_


In [20]:
venue_id = '4cefd14ef368b1f74bae87f7' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/4cefd14ef368b1f74bae87f7?client_id=O4YSARF1VKMD3JXMQCZYNAHTJCV3LHDB3DYMETHZXAAJMC0I&client_secret=0POWTMO1RCRDOJ4VGTKE4QJ5PCPNIRIBH5YCG4NX5LKW2V1W&oauth_token=C1TYPH4VKGU4T5LFZSTF3CC34N3R2OSQZHGYOEF5AFTDTDOY&v=20180604'

#### Send GET request for result


In [21]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4cefd14ef368b1f74bae87f7',
 'name': 'Pizza & Mortadella',
 'contact': {},
 'location': {'address': 'Via Cavour',
  'lat': 41.89387250170169,
  'lng': 12.49004491178546,
  'labeledLatLngs': [{'label': 'display',
    'lat': 41.89387250170169,
    'lng': 12.49004491178546}],
  'cc': 'IT',
  'city': 'Roma',
  'state': 'Lazio',
  'country': 'Italia',
  'formattedAddress': ['Via Cavour', 'Roma Lazio', 'Italia']},
 'canonicalUrl': 'https://foursquare.com/v/pizza--mortadella/4cefd14ef368b1f74bae87f7',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 7},
 'price': {'tier': 2, 'message': 'Moderate', 'currency': '€'},
 'likes': {'count': 6,
  'groups': [{'type': 'others',
    'count': 6,
    'items': [{'id': '126319800',
      'firstName': 'Lu

### B. Get the venue's overall rating


In [22]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.5
